In [16]:
import requests
import base64
import re
from PIL import Image

# Base URL for the API
BASE_URL = "http://localhost:8000"

def to_base64(data_bytes: bytes) -> str:
    """
    Encode bytes to a clean, padded Base64 string.
    Strips out any characters outside the Base64 alphabet
    and pads with '=' to ensure length is a multiple of 4.
    """
    encoded = base64.b64encode(data_bytes).decode('ascii')

    return encoded.removeprefix('b')

def test_root_endpoint():
    """Test the root endpoint"""
    response = requests.get(f"{BASE_URL}/")
    print("Root endpoint response:", response.json())

def test_file_encryption():
    """Test file encryption endpoint"""
    test_content = b"Hello, this is a test file!"
    encoded_content = to_base64(test_content)
    
    payload = {
        "file_content": encoded_content,
        "password": "testpassword",
        "algorithm": "AES-GCM",
        "kdf_type": "PBKDF2",
        "kdf_params": {},  # Empty dict as per schema
        "operation": "encrypt",
        "partial": False,
        "partial_text": "test",
        "partial_scope": "First Occurrence"
    }
    
    response = requests.post(f"{BASE_URL}/api/encrypt", json=payload)
    print("\nFile encryption response:", response.json())

def test_image_processing():
    """Test image processing endpoint with a real image"""
    # 1) Create a simple test image
    img = Image.new('RGB', (100, 100), color='white')
    img.save('test.png')

    # 2) Read and Base64‑encode the image (cleaned & padded)
    with open('processed_image.png', 'rb') as image_file:
        raw = image_file.read()
        encoded_image = to_base64(raw)

    # 3) Define regions to process (full image)
    regions = [
        {"left": 0, "top": 0, "width": 10, "height": 10},
        {"left": 10, "top": 10, "width": 10, "height": 10},
        
    ]
    import os
    # 4) Build the JSON payload
    payload = {
        "image_content": encoded_image,
        "algorithm": "ChaCha20",
        "key":  "00112233445566778899aabbccddeeff00112233445566778899aabbccddeeff", # 16 bytes key
        "nonce": "3f1a2b4c5d6e7f80",   # example 16‑char hex string
        "operation": "encrypt",
        "regions": regions
    }

    # 5) Send to the API
    response = requests.post(f"{BASE_URL}/api/image/process", json=payload)
    print("\nImage processing response:", response.json())

    # 6) Save the processed image if successful
    if response.status_code == 200:
        result = response.json()
        processed_data = base64.b64decode(result["processed_image"])
        output_filename = result.get("filename", "processed_test.png")
        with open(output_filename, "wb") as f:
            f.write(processed_data)
        print(f"Saved processed image as {output_filename}")

if __name__ == "__main__":
    print("Testing API endpoints...")
    test_root_endpoint()
    test_file_encryption()
    test_image_processing()


Testing API endpoints...
Root endpoint response: {'message': 'Welcome to SecureCrypt API'}

File encryption response: {'processed_content': 'DGPlsMzOny1N/kHsX15aS7OdQgDAnZFuNbEYOjLEIC5OLBhfAUb8tdwZ1Yk/8YTnNJp+ET1RxDwuVQ5ygf7rbDiuK640AUo=', 'filename': 'encrypted_file', 'mime_type': 'application/octet-stream'}

Image processing response: {'processed_image': 'iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAIAAAD/gAIDAAAA6ElEQVR4nO3QwQ3AIBDAsNL9dz5WIC+EZE8QZc3Mx5n/dsBLzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCswKzArMCjbP8QPFyqa8aAAAAABJRU5ErkJggg==', 'filename': 'processed_image.png'}
Saved processed image as processed_image.png


In [12]:


def test_auto_decrypt_image():
    """Test auto-decrypt image endpoint"""
    # 1) Read the previously processed image
    with open('processed_image.png', 'rb') as image_file:
        raw = image_file.read()
        encoded_image = to_base64(raw)

    # 2) Build the JSON payload
    payload = {
        "image_content": encoded_image,
        "algorithm": "ChaCha20",
        "key": "00112233445566778899aabbccddeeff00112233445566778899aabbccddeeff",  # same key used for encryption
        "nonce": "3f1a2b4c5d6e7f80"  # same nonce used for encryption
    }

    # 3) Send to the API
    response = requests.post(f"{BASE_URL}/api/image/auto-decrypt", json=payload)
    print("\nAuto-decrypt response:", response.json())

    # 4) Save the decrypted image if successful
    if response.status_code == 200:
        result = response.json()
        decrypted_data = base64.b64decode(result["processed_image"])
        output_filename = "decrypted_image.png"
        with open(output_filename, "wb") as f:
            f.write(decrypted_data)
        print(f"Saved decrypted image as {output_filename}")


test_auto_decrypt_image()  # Add the new test




Auto-decrypt response: {'processed_image': 'iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAIAAAD/gAIDAAAJEklEQVR4nO3X+TfU+wPH8ZlpiIZk1DC5MWSZJKYasnUJiUQqW00ptCqikhZLia8yXMtFtlFkCZEYu+FmFMmSIboxss00ESqJNPj+B/ec9w9zPufe8378AZ/zeT3f5/M+54Na/kfP8a173S2Iv+vSX1/VO2AYux+Hv4ePN5TJztXd//jtwRyT9rpiintlyEeZoWzmPz/qPwBTWM/t9/J8VJ1w+MBSgaqektRtesjm6x9IxN5ol40fq2vMJI3dlUsG1QIoRr7G64m8jPPnpyTcc9cOtOCUbvbfSwrNPODg5f2JSm2rR/3nnYwJdKZQNtyhTednEDNN7DqnQjUW2nApDR+Gu4vrFnxZhWcIBxJIm/xHy1C/SvWF29fpdYV/FaKlY7qFBwtsF+a3RO1u+m3F4l3vZKQPXuTQaUbdVaVb3zqssJEeZbgaXBs1JWp3SY2acMQX0/nZL65xFYsilPDEprZYXIb509pVfytcOvQ+NqiE7zaenOcjSMoX4md0AuXMegdWNT6oRfroRQubwJTqY8vIRoVKfzUR8vQY4e1T2PbJCmshyaTmcYVPDa/KzVp87CgV7yXmZMo+93pWEW2tg/aMium7ZYaN2nlGxWv1+foeYo5J6DGkt4gcpsG4U0miotk4y7VVYu7Ip5yf1jsY1KHFdy0ZmbmTjsyIH/f1BibDvfZ9GR1cFtzIlS8ePnVQhrMkXIhx9qGxtV3F5ryddE234OQjsEhvETmMZZH9uKNN6+oDajcI1DqDXHWKpkCczMtc6YkbHrFwfGQ7amb2bbIjH5NdEJHCMneWijTvDaF3pesYSNfIXWiVNSdL4E/TyBTNJ01IbxE5rHqy5ngWPYQUHp2wyd9NNeDXl+coIY+xo8rQvwG1RYtv+fkS9z0K95fv3A2a3QK